In [10]:
# Doc object
import spacy
nlp = spacy.load("en_core_web_sm")
doc = nlp("This is a sentence")
print(doc)

This is a sentence


#### Pipeline components

#### Tagger

In [11]:
for token in doc:
    print(f'{token.text}: ')
    print(f'[.tag] {token.tag}, part-of-speech tag Id')
    print(f'[.pos] {token.pos}, position in the sentence')
    print("")


This: 
[.tag] 15267657372422890137, part-of-speech tag Id
[.pos] 95, position in the sentence

is: 
[.tag] 13927759927860985106, part-of-speech tag Id
[.pos] 87, position in the sentence

a: 
[.tag] 15267657372422890137, part-of-speech tag Id
[.pos] 90, position in the sentence

sentence: 
[.tag] 15308085513773655218, part-of-speech tag Id
[.pos] 92, position in the sentence



#### Parser

In [12]:
for token in doc:
    print(f'{token.text}: ')
    print(f'[.head] {token.head if token.has_head() else None}, syntactic parent of the token, if hase one')
    print(f'[.dep] {token.dep}, syntactic dependency id')
    print(f'[.sent] {token.sent}, relative sentence span')
    print("")

This: 
[.head] is, syntactic parent of the token, if hase one
[.dep] 429, syntactic dependency id
[.sent] This is a sentence, relative sentence span

is: 
[.head] is, syntactic parent of the token, if hase one
[.dep] 8206900633647566924, syntactic dependency id
[.sent] This is a sentence, relative sentence span

a: 
[.head] sentence, syntactic parent of the token, if hase one
[.dep] 415, syntactic dependency id
[.sent] This is a sentence, relative sentence span

sentence: 
[.head] is, syntactic parent of the token, if hase one
[.dep] 404, syntactic dependency id
[.sent] This is a sentence, relative sentence span



#### Named Entity recognizer

In [13]:
for ent in doc.ents:
    print(f'{ent.text}: ')
    print(f'{ent}')

#### Text classifier

In [14]:
for c in doc.cats:
    print(f'{c}')

Because text categories are always very specific, the text classifier is not included in any of the trained pipelines by default. But you can use it to train your own system.